# **Data augmentation: RAG for sentences (15 classes)**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle
import numpy as np

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

!pip install openai

!pip install langchain
!pip install huggingface_hub
!pip install langchain-community langchain-core

!pip install -q langchain
!pip install -q torch
!pip install -q transformers
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q faiss-cpu
!pip install langchainhub

from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser
from langchain import hub
from langchain.chat_models import ChatOpenAI

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.6/375.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.5 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [ ]:
import openai

openai_api_key = "..." # put your OpenAI key here

## Load the data:

Load train set:

In [ ]:
f_in = open("drive/MyDrive/data/15-classes/train_set_sentences.pkl","rb")

data_train = pickle.load(f_in)

f_in.close()

Extract examples of specific classes:

In [ ]:
data_positive = [(data_train[i][1].split('\n\n'),data_train[i][2],data_train[i][3]) for i in range(len(data_train)) if data_train[i][2] == 0]
#data_positive = [(data_train[i][1].split('\n\n'),data_train[i][2],data_train[i][3]) for i in range(len(data_train)) if ((data_train[i][2] == 1) or (data_train[i][2] == 4) or (data_train[i][2] == 9) or (data_train[i][2] == 12) or (data_train[i][2] == 13))]
#data_positive = [(data_train[i][1].split('\n\n'),data_train[i][2],data_train[i][3]) for i in range(len(data_train)) if ((data_train[i][2] == 0) or (data_train[i][2] == 10))]
#data_positive = [(data_train[i][1].split('\n\n'),data_train[i][2],data_train[i][3]) for i in range(len(data_train)) if ((data_train[i][2] == 1) or (data_train[i][2] == 2) or (data_train[i][2] == 3) or (data_train[i][2] == 4) or (data_train[i][2] == 5) or (data_train[i][2] == 6))]
#data_positive = [(data_train[i][1].split('\n\n'),data_train[i][2],data_train[i][3]) for i in range(len(data_train)) if ((data_train[i][2] == 3) or (data_train[i][2] == 4) or (data_train[i][2] == 7) or (data_train[i][2] == 12) or (data_train[i][2] == 14))]

In [ ]:
data_positive[0:5]

[(['Questionnaire for traders, processors, converters of the end-product PHA',
   'Block “Introduction and ice breaker”',
   'OR Which materials/end products (mulch films, plant pots, twine, growth protection, greenhouses, nets, plant bags/growing pots made of PHA) do you produce in your company?',
   'Any outdoor application that needs to be durable is not a viable option for PHAs. PHAs degrade too easily.'],
  10,
  212),
 (['Questionnaire for traders, processors, converters of the end-product PHA',
   'Block “Introduction and ice breaker”',
   'OR Which materials/end products (mulch films, plant pots, twine, growth protection, greenhouses, nets, plant bags/growing pots made of PHA) do you produce in your company?',
   'Any outdoor application that needs to be durable is not a viable option for PHAs. PHAs degrade too easily. They break down really quickly.'],
  10,
  212),
 (['Questionnaire for traders, processors, converters of the end-product PHA',
   'Block “Introduction and ice b

Filter out paragraphs which are already present:

In [ ]:
data_train_positive = []
paras_id = []

for i in range(len(data_positive)):
  if data_positive[i][2] not in paras_id:
    data_train_positive.append((data_positive[i][0],data_positive[i][1]))
    paras_id.append(data_positive[i][2])

In [ ]:
len(data_train_positive)

15

In [ ]:
data_train_positive[0:3]

[(['Questionnaire for traders, processors, converters of the end-product PHA',
   'Block “Introduction and ice breaker”',
   'OR Which materials/end products (mulch films, plant pots, twine, growth protection, greenhouses, nets, plant bags/growing pots made of PHA) do you produce in your company?',
   'Any outdoor application that needs to be durable is not a viable option for PHAs. PHAs degrade too easily.'],
  10),
 (['Questionnaire for traders, processors, converters of the end-product PHA',
   'Block “Introduction and ice breaker”',
   'OR Which materials/end products (mulch films, plant pots, twine, growth protection, greenhouses, nets, plant bags/growing pots made of PHA) do you produce in your company?',
   'They get attacked by all organisms and can degrade within days or weeks. They lose their properties really quick if they get in touch with soil.'],
  10),
 (['Questionnaire for traders, processors, converters of the end-product PHA',
   'Block: Market opportunities',
   'How

## Retrieval augmented generation:

Use external data set from Hugging face:

In [ ]:
dataset_name = "Rahulrayudu/Agro_Dataset_QA_Final"

#page_content_column = "Crop"  # or any other column you're interested in
page_content_column = "Question"  # or any other column you're interested in

# Create a loader instance
loader = HuggingFaceDatasetLoader(dataset_name, page_content_column)

# Load the data
data = loader.load()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Final_Crop_QA_Dataset - Sheet1.csv:   0%|          | 0.00/709k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4153 [00:00<?, ? examples/s]

Use external data from the NoAW project:

In [ ]:
# Load list of files:
f = open("drive/MyDrive/data/RAG/filelist.txt", "r")

fileNames = []
for line in f:
    try:
      fName = line.strip()
      if fName != '':
          fileNames.append(fName)

    except ValueError:
        print('Invalid input:',line)

f.close()

In [ ]:
# Load data from multiple files:
#data = []

for file in fileNames:
  loader = TextLoader(file)

  data_file = loader.load()

  data.extend(data_file)

In [ ]:
len(data)

4179

Prepare the data:

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150) # fragment documents into chunks of 1000 characters

docs = text_splitter.split_documents(data)

In [ ]:
len(docs)

4930

Compute the embeddings:

In [ ]:
# Define the path to the pre-trained model you want to use
modelPath = "sentence-transformers/all-MiniLM-l6-v2"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

<ipython-input-16-a34a09b47e7d>:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Define database for storing embeddings:

In [ ]:
db = FAISS.from_documents(docs, embeddings)

Define query retriever:

In [ ]:
retriever = db.as_retriever(search_kwargs={"k": 3})

Define an LLM:

In [ ]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | hub.pull("rlm/rag-prompt")
    | ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=openai_api_key)
    | StrOutputParser()
)

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
<ipython-input-19-df3318f6b8f7>:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  | ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=openai_api_key)


## Generate new examples:

Experiment (augmentation) name:

In [ ]:
experiment = '106'

Generate new examples:

In [ ]:
new_training_set_text = []
new_training_set_labels = []

flag = False
for i in range(len(data_train_positive)):
  if data_train_positive[i][1] == 0: # Other (not pertinent)
    if "Questionnaire for " in data_train_positive[i][0][0]:
      request = "You are "+data_train_positive[i][0][0].split("Questionnaire for ")[1]+". "+data_train_positive[i][0][2]
      flag = True
  elif data_train_positive[i][1] == 1: # Stakeholders’ expectations > valorization/ PHA-applications
    request = "Give your expectations on the following question. "+data_train_positive[i][0][2]+" Make emphasis on valorization of PHA-applications"
    flag = True
  elif data_train_positive[i][1] == 2: # limitations and barriers > valorization /PHA-applications
    request = data_train_positive[i][0][2] + " Mention limitations and barriers in valorization of PHA-applications"
    flag = True
  elif data_train_positive[i][1] == 3: # market opportunities > PHA MO
    request = data_train_positive[i][0][2]+" Discuss market opportunities of PHA"
    flag = True
  elif data_train_positive[i][1] == 4: # market opportunities > PHA-Applications MO
    request = data_train_positive[i][0][2]+" Discuss market opportunities of PHA-applications"
    flag = True
  elif data_train_positive[i][1] == 5: # valorization > current structures
    if 'valoriz' in data_train_positive[i][0][2]:
      request = data_train_positive[i][0][2]
    else:
      request = data_train_positive[i][0][2]+" Make emphasis on valorization"
  elif data_train_positive[i][1] == 6: # company+Experts
    if "Questionnaire for " in data_train_positive[i][0][0]:
      request = "You are "+data_train_positive[i][0][0].split("Questionnaire for ")[1]+". Briefly describe your company and your position"
    flag = True
  elif data_train_positive[i][1] == 7: # limitations and barriers > Main issues and challenges for extracted/microbial protein
    request = data_train_positive[i][0][2] + " Focus on main issues and challenges for extracted/microbial protein"
    flag = True
  elif data_train_positive[i][1] == 8: # type of stream
    if "describe your company and your position within the company" in data_train_positive[i][0][2]:
      request = "Which type of residue stream do you deal with in your company? Give sowe examples"
    else:
      request = data_train_positive[i][0][2]+" Make emphasis on type of stream"
    flag = True
  elif data_train_positive[i][1] == 9: # Stakeholders’ expectations > PHA expectation
    request = "Give your expectations on the following question. "+data_train_positive[i][0][2]+" Make emphasis on PHA expectation"
    flag = True
  elif data_train_positive[i][1] == 10: # limitations and barriers > Main issues and challenges for PHA
    #request = data_train_positive[i][0][2] + " Focus on main issues and challenges for PHA"
    request = data_train_positive[i][0][2] + " Emphasise main issues and challenges for PHA"
    flag = True
  elif data_train_positive[i][1] == 11: # market opportunities > MP MO
    request = data_train_positive[i][0][2]+" Discuss market opportunities of microbial proteins (MP)"
    flag = True
  elif data_train_positive[i][1] == 12: # Stakeholders’ expectations > MP
    request = "Give your expectations on the following question. "+data_train_positive[i][0][2]+" Make emphasis on microbial protein (MP)"
    flag = True
  elif data_train_positive[i][1] == 13: # valorization > satisfaction
    #request = data_train_positive[i][0][2]
    request = data_train_positive[i][0][2]+" Generate another one"
    flag = True
  elif data_train_positive[i][1] == 14: # valorization > advantages
    if 'How do you currently valorize' in data_train_positive[i][0][2]:
      request = data_train_positive[i][0][2]+" Emphasize advantages"
    else:
      request = data_train_positive[i][0][2]
    flag = True
  if flag:
    response = rag_chain.invoke(request)
    if not "I don't know" in response:
      # 'response' split on sentences:
      response_sents = sent_tokenize(response)
      for j in range(len(response_sents)):
        # Add previous and next sentences if they have same code:
        if j==0: # the 1st sentence of paragraph
          if len(response_sents)>1:
            # add current and next sentences:
            segment = response_sents[j]+' '+response_sents[j+1]
          else:
            # add current sentence only:
            segment = response_sents[j]
        elif j<len(response_sents)-1: # i-th sentence of paragraph
          # add previous, current and next sentences:
          segment = response_sents[j-1]+' '+response_sents[j]+' '+response_sents[j+1]
        else: # last sentence of paragraph
          # add previous and current sentences:
          segment = response_sents[j-1]+' '+response_sents[j]
        # Memorise the result:
        new_example = data_train_positive[i][0][0]+"\n\n"+data_train_positive[i][0][1]+"\n\n"+data_train_positive[i][0][2]+"\n\n"+segment.strip()
        if new_example not in new_training_set_text:
          new_training_set_text.append(new_example)
          new_training_set_labels.append(data_train_positive[i][1])
    flag = False
  if i % 10 == 0:
    print("Generating",i,"segment")
    print(request)

Generating 0 segment
OR Which materials/end products (mulch films, plant pots, twine, growth protection, greenhouses, nets, plant bags/growing pots made of PHA) do you produce in your company? Emphasise main issues and challenges for PHA
Generating 10 segment
Please describe the challenges regarding the application of mulch films, horticultural pots, twines, growth protectors, greenhouses, nets, grow/seedling bags from PHA Emphasise main issues and challenges for PHA


Generated examples:

In [ ]:
new_training_set_text[:5]

['Questionnaire for arable farmers/ horticulture businesses/ plant production / food processors (= suppliers of residual streams)\n\nBlock “strengths and challenges”\n\nWhich are the strengths and advantages in the current valorization of by-products of apple-/tomato-/grape-/potato processing/brewer’s grains?\n\nStrengths and advantages in the current valorization of by-products of apple processing include the ability to maintain natural nutrient supply, low logistic costs, and alignment with circularity/sustainability principles. Some weaknesses include the cost related to collection and transportation of the by-products, as well as the expense of machines used for collection.',
 'Questionnaire for arable farmers/ horticulture businesses/ plant production / food processors (= suppliers of residual streams)\n\nBlock “strengths and challenges”\n\nWhich are the strengths and advantages in the current valorization of by-products of apple-/tomato-/grape-/potato processing/brewer’s grains?\

Create new segments:

In [ ]:
data_new = [(-1,new_training_set_text[i],new_training_set_labels[i]) for i in range(len(new_training_set_text))]

data_augmented = data_train + data_new

Some stats:

In [ ]:
print('Class 0:', len([i for i in range(len(data_new)) if data_new[i][2]==0]))
print('Class 1:', len([i for i in range(len(data_new)) if data_new[i][2]==1]))
print('Class 2:', len([i for i in range(len(data_new)) if data_new[i][2]==2]))
print('Class 3:', len([i for i in range(len(data_new)) if data_new[i][2]==3]))
print('Class 4:', len([i for i in range(len(data_new)) if data_new[i][2]==4]))
print('Class 5:', len([i for i in range(len(data_new)) if data_new[i][2]==5]))
print('Class 6:', len([i for i in range(len(data_new)) if data_new[i][2]==6]))
print('Class 7:', len([i for i in range(len(data_new)) if data_new[i][2]==7]))
print('Class 8:', len([i for i in range(len(data_new)) if data_new[i][2]==8]))
print('Class 9:', len([i for i in range(len(data_new)) if data_new[i][2]==9]))
print('Class 10:', len([i for i in range(len(data_new)) if data_new[i][2]==10]))
print('Class 11:', len([i for i in range(len(data_new)) if data_new[i][2]==11]))
print('Class 12:', len([i for i in range(len(data_new)) if data_new[i][2]==12]))
print('Class 13:', len([i for i in range(len(data_new)) if data_new[i][2]==13]))
print('Class 14:', len([i for i in range(len(data_new)) if data_new[i][2]==14]))
print('Total:', len(data_new))

Class 0: 463
Class 1: 0
Class 2: 0
Class 3: 0
Class 4: 0
Class 5: 0
Class 6: 0
Class 7: 0
Class 8: 0
Class 9: 0
Class 10: 54
Class 11: 0
Class 12: 0
Class 13: 0
Class 14: 0
Total: 517


In [ ]:
len(data_augmented)

2056

Save results:

In [ ]:
f_out = open("drive/MyDrive/data/15-classes/train_set_sentences_augm-"+experiment+".pkl","wb")

pickle.dump(data_augmented,f_out)

f_out.close()